Quick analysis

In [ ]:
#!/usr/bin/env python3
import os
import json
import h5py
import argparse
import pandas as pd
import numpy as np
import tinydb as db
from tinydb.storages import MemoryStorage
import matplotlib.pyplot as plt
# plt.style.use('../clint.mpl')
from matplotlib.colors import LogNorm

import boost_histogram as bh
import pickle as pl

from pygama import DataGroup
import pygama.lh5 as lh5
import pygama.analysis.histograms as pgh
import pygama.analysis.peak_fitting as pgf
import cage_utils
plt.rcParams['figure.figsize'] = [14, 10]

In [ ]:
run = 70 #64
user = False
hit =True
cal = True
etype = 'trapEftp_cal'

corr_DCR=True
corr_AoE=True
corr_ToE=True

norm = True
cut = True

dsp_list = ['trapEftp', 'trapEftp_cal', 'bl','bl_sig', 'AoE', 'dcr', "tp_0", "tp_10", "tp_20", "tp_30", "tp_40", "tp_50", "tp_60", "tp_70", "tp_80", "tp_90", "tp_96", 'tp_max', 'ToE']

df_raw, runtype, rt_min, radius, angle_det, rotary = cage_utils.getDataFrame(run, user=user, hit=hit, cal=cal, dsp_list=dsp_list)
# use baseline cut
if run <79:
    bl_cut_lo, bl_cut_hi = 9150,9320
if run>79 and run <117:
    bl_cut_lo, bl_cut_hi = 8500, 10000
if run>=117:
    bl_cut_lo, bl_cut_hi = 9700, 976
# only do basic bl cut before correcting DCR, AoE and ToE.
# If want to add extra cut while correcting AoE or ToE, use cut specifically in their mode_hist() call
df = df_raw.query(f'bl > {bl_cut_lo} and bl < {bl_cut_hi}').copy()

if 'dcr' in dsp_list and corr_DCR==True:
    if run>57:
        const, offset, err = cage_utils.corrDCR(df, etype, e_bins=300, elo=0, ehi=6000, dcr_fit_lo=-30, dcr_fit_hi=40)
        df['dcr_plot'] = df['dcr']-offset + ((-1*const))*df[etype]
    elif corr_DCR==True and run<57:
        const = const = 0.0011
        df['dcr_plot'] = df['dcr'] - const*df['trapEftp']
else:
    df['dcr_plot'] = df['dcr']
    
if 'AoE' in dsp_list and corr_AoE==True:
    AoE_mode = cage_utils.mode_hist(df, param='AoE', a_bins=1000, alo=0.005, ahi=0.075, cut=False, cut_str='')
    df['AoE_plot'] = df['AoE'] - AoE_mode
else:
    df['AoE_plot'] = df['AoE']

if 'ToE' in dsp_list and corr_ToE==True:
    ToE_mode = cage_utils.mode_hist(df, param='ToE', a_bins=1000, alo=0.30, ahi=0.45, cut=False, cut_str='')
    df['ToE_plot'] = df['ToE'] - ToE_mode
else:
    df['ToE_plot'] = df['ToE']

In [ ]:
# Create all the timepoint parameters!

df['tp0010'] = df['tp_10'] - df['tp_0']
df['tp0020'] = df['tp_20'] - df['tp_0']
df['tp0030'] = df['tp_30'] - df['tp_0']
df['tp0040'] = df['tp_40'] - df['tp_0']
df['tp0050'] = df['tp_50'] - df['tp_0']
df['tp0060'] = df['tp_60'] - df['tp_0']
df['tp0070'] = df['tp_70'] - df['tp_0']
df['tp0080'] = df['tp_80'] - df['tp_0']
df['tp0090'] = df['tp_90'] - df['tp_0']
df['tp0096'] = df['tp_96'] - df['tp_0']

df['tp1020'] = df['tp_20'] - df['tp_10']
df['tp1030'] = df['tp_30'] - df['tp_10']
df['tp1040'] = df['tp_40'] - df['tp_10']
df['tp1050'] = df['tp_50'] - df['tp_10']
df['tp1060'] = df['tp_60'] - df['tp_10']
df['tp1070'] = df['tp_70'] - df['tp_10']
df['tp1080'] = df['tp_80'] - df['tp_10']
df['tp1090'] = df['tp_90'] - df['tp_10']
df['tp1096'] = df['tp_96'] - df['tp_10']

df['tp2030'] = df['tp_30'] - df['tp_20']
df['tp2040'] = df['tp_40'] - df['tp_20']
df['tp2050'] = df['tp_50'] - df['tp_20']
df['tp2060'] = df['tp_60'] - df['tp_20']
df['tp2070'] = df['tp_70'] - df['tp_20']
df['tp2080'] = df['tp_80'] - df['tp_20']
df['tp2090'] = df['tp_90'] - df['tp_20']
df['tp2096'] = df['tp_96'] - df['tp_20']

df['tp3040'] = df['tp_40'] - df['tp_30']
df['tp3050'] = df['tp_50'] - df['tp_30']
df['tp3060'] = df['tp_60'] - df['tp_30']
df['tp3070'] = df['tp_70'] - df['tp_30']
df['tp3080'] = df['tp_80'] - df['tp_30']
df['tp3090'] = df['tp_90'] - df['tp_30']
df['tp3096'] = df['tp_96'] - df['tp_30']

df['tp4050'] = df['tp_50'] - df['tp_40']
df['tp4060'] = df['tp_60'] - df['tp_40']
df['tp4070'] = df['tp_70'] - df['tp_40']
df['tp4080'] = df['tp_80'] - df['tp_40']
df['tp4090'] = df['tp_90'] - df['tp_40']
df['tp4096'] = df['tp_96'] - df['tp_40']

df['tp4050'] = df['tp_50'] - df['tp_40']
df['tp4060'] = df['tp_60'] - df['tp_40']
df['tp4070'] = df['tp_70'] - df['tp_40']
df['tp4080'] = df['tp_80'] - df['tp_40']
df['tp4090'] = df['tp_90'] - df['tp_40']
df['tp4096'] = df['tp_96'] - df['tp_40']

df['tp5060'] = df['tp_60'] - df['tp_50']
df['tp5070'] = df['tp_70'] - df['tp_50']
df['tp5080'] = df['tp_80'] - df['tp_50']
df['tp5090'] = df['tp_90'] - df['tp_50']
df['tp5096'] = df['tp_96'] - df['tp_50']

df['tp6070'] = df['tp_70'] - df['tp_60']
df['tp6080'] = df['tp_80'] - df['tp_60']
df['tp6090'] = df['tp_90'] - df['tp_60']
df['tp6096'] = df['tp_96'] - df['tp_60']

df['tp7080'] = df['tp_80'] - df['tp_70']
df['tp7090'] = df['tp_90'] - df['tp_70']
df['tp7096'] = df['tp_96'] - df['tp_70']

df['tp8090'] = df['tp_90'] - df['tp_80']
df['tp8096'] = df['tp_96'] - df['tp_80']

df['tp9096'] = df['tp_96'] - df['tp_90']

In [ ]:
# Create a cut if appropriate
eCut_lo, eCut_hi = 500, 6000

cut_str = f'trapEftp_cal > {eCut_lo} and trapEftp_cal < {eCut_hi}'

if cut == True:
    print(f'Using cut: {cut_str}')
    df_cut = df.query(cut_str).copy()
else:
    df_cut = df

In [ ]:
if norm==True:
    rt = np.array([(1/rt_min)])
    wts = np.repeat(rt, len(df_cut[etype]))
else:
    rt = np.array([(1/1.)])
    wts = np.repeat(rt, len(df_cut[etype]))

In [ ]:
if run>=36 and run<117:
    dlo, dhi = -40, 170
    d_bins = 200
elif run>=117:
    # dlo, dhi, dpb = -20., 40, 0.1
    dlo, dhi = -40, 170
    d_bins = 200
fig, ax = plt.subplots()   
tlo, thi, tpb = 0, 400, 10
t_bins = int((thi-tlo)/tpb)
fig.suptitle(f'DCR vs TP', horizontalalignment='center', fontsize=16)
tpVdcr_hist_norm, xedges, yedges = np.histogram2d(df_cut['tp0010'], df_cut['dcr_plot'], bins=[t_bins, d_bins], range=([tlo, thi], [dlo, dhi]), weights=wts)
X, Y = np.mgrid[tlo:thi:t_bins*1j, dlo:dhi:d_bins*1j]

pcm = plt.pcolormesh(X, Y, tpVdcr_hist_norm, norm=LogNorm(0.001, 10))
cb = plt.colorbar()
cb.set_label("counts/min", ha = 'right', va='center', rotation=270, fontsize=14)
cb.ax.tick_params(labelsize=12)
ax.set_xlabel('TP 0-10 (arb)', fontsize=16)
ax.set_ylabel('DCR (arb)', fontsize=16)
plt.setp(ax.get_xticklabels(), fontsize=12)
plt.setp(ax.get_yticklabels(), fontsize=14)
plt.show()

In [ ]:
tp_hi = [10, 20, 30, 40, 50, 60, 70, 80, 90, 96]
tp_param_list = []
for t in tp_hi:
    tp_param_list.append(f'tp00{t}')
print(tp_param_list)

for tp_param in tp_param_list:
    if run>=36 and run<117:
        dlo, dhi = -40, 170
        d_bins = 200
    elif run>=117:
        # dlo, dhi, dpb = -20., 40, 0.1
        dlo, dhi = -40, 170
        d_bins = 200
    fig, ax = plt.subplots()   
    tlo, thi, tpb = 0, 3000, 10
    t_bins = int((thi-tlo)/tpb)
    fig.suptitle(f'\nDCR vs {tp_param}', horizontalalignment='center', fontsize=16)
    tpVdcr_hist_norm, xedges, yedges = np.histogram2d(df_cut[f'{tp_param}'], df_cut['dcr_plot'], bins=[t_bins, d_bins], range=([tlo, thi], [dlo, dhi]), weights=wts)
    X, Y = np.mgrid[tlo:thi:t_bins*1j, dlo:dhi:d_bins*1j]
    
    pcm = plt.pcolormesh(X, Y, tpVdcr_hist_norm, norm=LogNorm(0.001, 10))
    cb = plt.colorbar()
    cb.set_label("counts/min", ha = 'right', va='center', rotation=270, fontsize=14)
    cb.ax.tick_params(labelsize=12)
    ax.set_xlabel(f'{tp_param}(arb)', fontsize=16)
    ax.set_ylabel('DCR (arb)', fontsize=16)
    plt.setp(ax.get_xticklabels(), fontsize=12)
    plt.setp(ax.get_yticklabels(), fontsize=14)
    plt.show()

In [ ]:
tp_hi = [20, 30, 40, 50, 60, 70, 80, 90, 96]
tp_param_list = []
for t in tp_hi:
    tp_param_list.append(f'tp10{t}')
print(tp_param_list)

for tp_param in tp_param_list:
    if run>=36 and run<117:
        dlo, dhi = -40, 170
        d_bins = 200
    elif run>=117:
        # dlo, dhi, dpb = -20., 40, 0.1
        dlo, dhi = -40, 170
        d_bins = 200
    fig, ax = plt.subplots()   
    tlo, thi, tpb = 0, 500, 10
    t_bins = int((thi-tlo)/tpb)
    fig.suptitle(f'\nDCR vs {tp_param}', horizontalalignment='center', fontsize=16)
    tpVdcr_hist_norm, xedges, yedges = np.histogram2d(df_cut[f'{tp_param}'], df_cut['dcr_plot'], bins=[t_bins, d_bins], range=([tlo, thi], [dlo, dhi]), weights=wts)
    X, Y = np.mgrid[tlo:thi:t_bins*1j, dlo:dhi:d_bins*1j]
    
    pcm = plt.pcolormesh(X, Y, tpVdcr_hist_norm, norm=LogNorm(0.001, 10))
    cb = plt.colorbar()
    cb.set_label("counts/min", ha = 'right', va='center', rotation=270, fontsize=14)
    cb.ax.tick_params(labelsize=12)
    ax.set_xlabel(f'{tp_param}(arb)', fontsize=16)
    ax.set_ylabel('DCR (arb)', fontsize=16)
    plt.setp(ax.get_xticklabels(), fontsize=12)
    plt.setp(ax.get_yticklabels(), fontsize=14)
    plt.show()

In [ ]:
tp_hi = [30, 40, 50, 60, 70, 80, 90, 96]
tp_param_list = []
for t in tp_hi:
    tp_param_list.append(f'tp20{t}')
print(tp_param_list)

for tp_param in tp_param_list:
    if run>=36 and run<117:
        dlo, dhi = -40, 170
        d_bins = 200
    elif run>=117:
        # dlo, dhi, dpb = -20., 40, 0.1
        dlo, dhi = -40, 170
        d_bins = 200
    fig, ax = plt.subplots()   
    tlo, thi, tpb = 0, 500, 10
    t_bins = int((thi-tlo)/tpb)
    fig.suptitle(f'\nDCR vs {tp_param}', horizontalalignment='center', fontsize=16)
    tpVdcr_hist_norm, xedges, yedges = np.histogram2d(df_cut[f'{tp_param}'], df_cut['dcr_plot'], bins=[t_bins, d_bins], range=([tlo, thi], [dlo, dhi]), weights=wts)
    X, Y = np.mgrid[tlo:thi:t_bins*1j, dlo:dhi:d_bins*1j]
    
    pcm = plt.pcolormesh(X, Y, tpVdcr_hist_norm, norm=LogNorm(0.001, 10))
    cb = plt.colorbar()
    cb.set_label("counts/min", ha = 'right', va='center', rotation=270, fontsize=14)
    cb.ax.tick_params(labelsize=12)
    ax.set_xlabel(f'{tp_param}(arb)', fontsize=16)
    ax.set_ylabel('DCR (arb)', fontsize=16)
    plt.setp(ax.get_xticklabels(), fontsize=12)
    plt.setp(ax.get_yticklabels(), fontsize=14)
    plt.show()

In [ ]:
tp_hi = [40, 50, 60, 70, 80, 90, 96]
tp_param_list = []
for t in tp_hi:
    tp_param_list.append(f'tp30{t}')
print(tp_param_list)

for tp_param in tp_param_list:
    if run>=36 and run<117:
        dlo, dhi = -40, 170
        d_bins = 200
    elif run>=117:
        # dlo, dhi, dpb = -20., 40, 0.1
        dlo, dhi = -40, 170
        d_bins = 200
    fig, ax = plt.subplots()   
    tlo, thi, tpb = 0, 500, 10
    t_bins = int((thi-tlo)/tpb)
    fig.suptitle(f'\nDCR vs {tp_param}', horizontalalignment='center', fontsize=16)
    tpVdcr_hist_norm, xedges, yedges = np.histogram2d(df_cut[f'{tp_param}'], df_cut['dcr_plot'], bins=[t_bins, d_bins], range=([tlo, thi], [dlo, dhi]), weights=wts)
    X, Y = np.mgrid[tlo:thi:t_bins*1j, dlo:dhi:d_bins*1j]
    
    pcm = plt.pcolormesh(X, Y, tpVdcr_hist_norm, norm=LogNorm(0.001, 10))
    cb = plt.colorbar()
    cb.set_label("counts/min", ha = 'right', va='center', rotation=270, fontsize=14)
    cb.ax.tick_params(labelsize=12)
    ax.set_xlabel(f'{tp_param}(arb)', fontsize=16)
    ax.set_ylabel('DCR (arb)', fontsize=16)
    plt.setp(ax.get_xticklabels(), fontsize=12)
    plt.setp(ax.get_yticklabels(), fontsize=14)
    plt.show()

In [ ]:
tp_hi = [50, 60, 70, 80, 90, 96]
tp_param_list = []
for t in tp_hi:
    tp_param_list.append(f'tp40{t}')
print(tp_param_list)

for tp_param in tp_param_list:
    if run>=36 and run<117:
        dlo, dhi = -40, 170
        d_bins = 200
    elif run>=117:
        # dlo, dhi, dpb = -20., 40, 0.1
        dlo, dhi = -40, 170
        d_bins = 200
    fig, ax = plt.subplots()   
    tlo, thi, tpb = 0, 500, 10
    t_bins = int((thi-tlo)/tpb)
    fig.suptitle(f'\nDCR vs {tp_param}', horizontalalignment='center', fontsize=16)
    tpVdcr_hist_norm, xedges, yedges = np.histogram2d(df_cut[f'{tp_param}'], df_cut['dcr_plot'], bins=[t_bins, d_bins], range=([tlo, thi], [dlo, dhi]), weights=wts)
    X, Y = np.mgrid[tlo:thi:t_bins*1j, dlo:dhi:d_bins*1j]
    
    pcm = plt.pcolormesh(X, Y, tpVdcr_hist_norm, norm=LogNorm(0.001, 10))
    cb = plt.colorbar()
    cb.set_label("counts/min", ha = 'right', va='center', rotation=270, fontsize=14)
    cb.ax.tick_params(labelsize=12)
    ax.set_xlabel(f'{tp_param}(arb)', fontsize=16)
    ax.set_ylabel('DCR (arb)', fontsize=16)
    plt.setp(ax.get_xticklabels(), fontsize=12)
    plt.setp(ax.get_yticklabels(), fontsize=14)
    plt.show()

In [ ]:
tp_hi = [60, 70, 80, 90, 96]
tp_param_list = []
for t in tp_hi:
    tp_param_list.append(f'tp50{t}')
print(tp_param_list)

for tp_param in tp_param_list:
    if run>=36 and run<117:
        dlo, dhi = -40, 170
        d_bins = 200
    elif run>=117:
        # dlo, dhi, dpb = -20., 40, 0.1
        dlo, dhi = -40, 170
        d_bins = 200
    fig, ax = plt.subplots()   
    tlo, thi, tpb = 0, 500, 10
    t_bins = int((thi-tlo)/tpb)
    fig.suptitle(f'\nDCR vs {tp_param}', horizontalalignment='center', fontsize=16)
    tpVdcr_hist_norm, xedges, yedges = np.histogram2d(df_cut[f'{tp_param}'], df_cut['dcr_plot'], bins=[t_bins, d_bins], range=([tlo, thi], [dlo, dhi]), weights=wts)
    X, Y = np.mgrid[tlo:thi:t_bins*1j, dlo:dhi:d_bins*1j]
    
    pcm = plt.pcolormesh(X, Y, tpVdcr_hist_norm, norm=LogNorm(0.001, 10))
    cb = plt.colorbar()
    cb.set_label("counts/min", ha = 'right', va='center', rotation=270, fontsize=14)
    cb.ax.tick_params(labelsize=12)
    ax.set_xlabel(f'{tp_param}(arb)', fontsize=16)
    ax.set_ylabel('DCR (arb)', fontsize=16)
    plt.setp(ax.get_xticklabels(), fontsize=12)
    plt.setp(ax.get_yticklabels(), fontsize=14)
    plt.show()

In [ ]:
tp_hi = [70, 80, 90, 96]
tp_param_list = []
for t in tp_hi:
    tp_param_list.append(f'tp60{t}')
print(tp_param_list)

for tp_param in tp_param_list:
    if run>=36 and run<117:
        dlo, dhi = -40, 170
        d_bins = 200
    elif run>=117:
        # dlo, dhi, dpb = -20., 40, 0.1
        dlo, dhi = -40, 170
        d_bins = 200
    fig, ax = plt.subplots()   
    tlo, thi, tpb = 0, 500, 10
    t_bins = int((thi-tlo)/tpb)
    fig.suptitle(f'\nDCR vs {tp_param}', horizontalalignment='center', fontsize=16)
    tpVdcr_hist_norm, xedges, yedges = np.histogram2d(df_cut[f'{tp_param}'], df_cut['dcr_plot'], bins=[t_bins, d_bins], range=([tlo, thi], [dlo, dhi]), weights=wts)
    X, Y = np.mgrid[tlo:thi:t_bins*1j, dlo:dhi:d_bins*1j]
    
    pcm = plt.pcolormesh(X, Y, tpVdcr_hist_norm, norm=LogNorm(0.001, 10))
    cb = plt.colorbar()
    cb.set_label("counts/min", ha = 'right', va='center', rotation=270, fontsize=14)
    cb.ax.tick_params(labelsize=12)
    ax.set_xlabel(f'{tp_param}(arb)', fontsize=16)
    ax.set_ylabel('DCR (arb)', fontsize=16)
    plt.setp(ax.get_xticklabels(), fontsize=12)
    plt.setp(ax.get_yticklabels(), fontsize=14)
    plt.show()

In [ ]:
tp_hi = [80, 90, 96]
tp_param_list = []
for t in tp_hi:
    tp_param_list.append(f'tp70{t}')
print(tp_param_list)

for tp_param in tp_param_list:
    if run>=36 and run<117:
        dlo, dhi = -40, 170
        d_bins = 200
    elif run>=117:
        # dlo, dhi, dpb = -20., 40, 0.1
        dlo, dhi = -40, 170
        d_bins = 200
    fig, ax = plt.subplots()   
    tlo, thi, tpb = 0, 500, 10
    t_bins = int((thi-tlo)/tpb)
    fig.suptitle(f'\nDCR vs {tp_param}', horizontalalignment='center', fontsize=16)
    tpVdcr_hist_norm, xedges, yedges = np.histogram2d(df_cut[f'{tp_param}'], df_cut['dcr_plot'], bins=[t_bins, d_bins], range=([tlo, thi], [dlo, dhi]), weights=wts)
    X, Y = np.mgrid[tlo:thi:t_bins*1j, dlo:dhi:d_bins*1j]
    
    pcm = plt.pcolormesh(X, Y, tpVdcr_hist_norm, norm=LogNorm(0.001, 10))
    cb = plt.colorbar()
    cb.set_label("counts/min", ha = 'right', va='center', rotation=270, fontsize=14)
    cb.ax.tick_params(labelsize=12)
    ax.set_xlabel(f'{tp_param}(arb)', fontsize=16)
    ax.set_ylabel('DCR (arb)', fontsize=16)
    plt.setp(ax.get_xticklabels(), fontsize=12)
    plt.setp(ax.get_yticklabels(), fontsize=14)
    plt.show()

In [ ]:
tp_hi = [90, 96]
tp_param_list = []
for t in tp_hi:
    tp_param_list.append(f'tp80{t}')
print(tp_param_list)

for tp_param in tp_param_list:
    if run>=36 and run<117:
        dlo, dhi = -40, 170
        d_bins = 200
    elif run>=117:
        # dlo, dhi, dpb = -20., 40, 0.1
        dlo, dhi = -40, 170
        d_bins = 200
    fig, ax = plt.subplots()   
    tlo, thi, tpb = 0, 500, 10
    t_bins = int((thi-tlo)/tpb)
    fig.suptitle(f'\nDCR vs {tp_param}', horizontalalignment='center', fontsize=16)
    tpVdcr_hist_norm, xedges, yedges = np.histogram2d(df_cut[f'{tp_param}'], df_cut['dcr_plot'], bins=[t_bins, d_bins], range=([tlo, thi], [dlo, dhi]), weights=wts)
    X, Y = np.mgrid[tlo:thi:t_bins*1j, dlo:dhi:d_bins*1j]
    
    pcm = plt.pcolormesh(X, Y, tpVdcr_hist_norm, norm=LogNorm(0.001, 10))
    cb = plt.colorbar()
    cb.set_label("counts/min", ha = 'right', va='center', rotation=270, fontsize=14)
    cb.ax.tick_params(labelsize=12)
    ax.set_xlabel(f'{tp_param}(arb)', fontsize=16)
    ax.set_ylabel('DCR (arb)', fontsize=16)
    plt.setp(ax.get_xticklabels(), fontsize=12)
    plt.setp(ax.get_yticklabels(), fontsize=14)
    plt.show()

In [ ]:

tp_param_list = ['tp9096']

for tp_param in tp_param_list:
    if run>=36 and run<117:
        dlo, dhi = -40, 170
        d_bins = 200
    elif run>=117:
        # dlo, dhi, dpb = -20., 40, 0.1
        dlo, dhi = -40, 170
        d_bins = 200
    fig, ax = plt.subplots()   
    tlo, thi, tpb = 0, 500, 10
    t_bins = int((thi-tlo)/tpb)
    fig.suptitle(f'\nDCR vs {tp_param}', horizontalalignment='center', fontsize=16)
    tpVdcr_hist_norm, xedges, yedges = np.histogram2d(df_cut[f'{tp_param}'], df_cut['dcr_plot'], bins=[t_bins, d_bins], range=([tlo, thi], [dlo, dhi]), weights=wts)
    X, Y = np.mgrid[tlo:thi:t_bins*1j, dlo:dhi:d_bins*1j]
    
    pcm = plt.pcolormesh(X, Y, tpVdcr_hist_norm, norm=LogNorm(0.001, 10))
    cb = plt.colorbar()
    cb.set_label("counts/min", ha = 'right', va='center', rotation=270, fontsize=14)
    cb.ax.tick_params(labelsize=12)
    ax.set_xlabel(f'{tp_param}(arb)', fontsize=16)
    ax.set_ylabel('DCR (arb)', fontsize=16)
    plt.setp(ax.get_xticklabels(), fontsize=12)
    plt.setp(ax.get_yticklabels(), fontsize=14)
    plt.show()

In [ ]:
# Make alpha cut
dcrCut_lo, dcrCut_hi = 20, 150
tCut_lo, tCut_hi = 0, 90 
alpha_cut = f'dcr > {dcrCut_lo} and dcr < {dcrCut_hi} and tp1020 > {tCut_lo} and tp1020 < {tCut_hi}'

df_alphaCut = df.query(alpha_cut).copy()


wts_alphaCut = np.repeat(rt, len(df_alphaCut[etype]))

In [ ]:
# normalized by runtime
fig, ax = plt.subplots()
alo, ahi, apb = 0.0, 0.09, 0.0001
if run>=36:
    alo, ahi, apb = 0.005, 0.075, 0.0001
if run>=117:
    alo, ahi, apb = 0.0, 0.15, 0.00015

if corr_AoE==True:
    alo, ahi, apb= -0.03, 0.03, 0.0005
    
elo, ehi, epb = 500, 6000, 1

nbx = int((ehi-elo)/epb)
nby = int((ahi-alo)/apb)

fig.suptitle(f'A/E vs Energy', horizontalalignment='center', fontsize=16)

cut_aoe_hist_norm, xedges, yedges = np.histogram2d(df_alphaCut[etype], df_alphaCut['AoE_plot'], bins=[nbx, nby], range=([elo, ehi], [alo, ahi]), weights=wts_alphaCut)
X, Y = np.mgrid[elo:ehi:nbx*1j, alo:ahi:nby*1j]

#             aoe_hist_norm = np.divide(aoe_hist, (rt_min))

pcm = plt.pcolormesh(X, Y, cut_aoe_hist_norm, norm=LogNorm(0.001, 0.1)) #0.002, 0.2

cb = plt.colorbar()
cb.set_label("counts/min", ha = 'right', va='center', rotation=270, fontsize=14)
cb.ax.tick_params(labelsize=12)
ax.set_xlabel(f'Energy (kev)', fontsize=16)
ax.set_ylabel('A/E (arb)', fontsize=16)
plt.setp(ax.get_xticklabels(), fontsize=14)
plt.setp(ax.get_yticklabels(), fontsize=14)


ax.text(0.95, 0.83, f'r = {radius} mm \ntheta = {angle_det} deg', verticalalignment='bottom',
            horizontalalignment='right', transform=ax.transAxes, color='green', fontsize=14, bbox={'facecolor': 'white', 'alpha': 0.5, 'pad': 10})

# plt.legend()
plt.title(f'\n{runtype} run {run}, {rt_min:.2f} mins', fontsize=12)
plt.tight_layout()

In [ ]:
# normalized by runtime
fig, ax = plt.subplots()
alo, ahi, apb = 0.0, 0.09, 0.0001
if run>=36:
    alo, ahi, apb = 0.005, 0.075, 0.0001
if run>=117:
    alo, ahi, apb = 0.0, 0.15, 0.00015

if corr_AoE==True:
    alo, ahi, apb= -0.03, 0.03, 0.0005
    
elo, ehi, epb = 500, 6000, 1

nbx = int((ehi-elo)/epb)
nby = int((ahi-alo)/apb)

fig.suptitle(f'A/E vs Energy', horizontalalignment='center', fontsize=16)

aoe_hist_norm, xedges, yedges = np.histogram2d(df_cut[etype], df_cut['AoE_plot'], bins=[nbx, nby], range=([elo, ehi], [alo, ahi]), weights=wts)
X, Y = np.mgrid[elo:ehi:nbx*1j, alo:ahi:nby*1j]

#             aoe_hist_norm = np.divide(aoe_hist, (rt_min))

pcm = plt.pcolormesh(X, Y, aoe_hist_norm, norm=LogNorm(0.001, 0.1)) #0.002, 0.2

cb = plt.colorbar()
cb.set_label("counts/min", ha = 'right', va='center', rotation=270, fontsize=14)
cb.ax.tick_params(labelsize=12)
ax.set_xlabel(f'Energy (kev)', fontsize=16)
ax.set_ylabel('A/E (arb)', fontsize=16)
plt.setp(ax.get_xticklabels(), fontsize=14)
plt.setp(ax.get_yticklabels(), fontsize=14)


ax.text(0.95, 0.83, f'r = {radius} mm \ntheta = {angle_det} deg', verticalalignment='bottom',
            horizontalalignment='right', transform=ax.transAxes, color='green', fontsize=14, bbox={'facecolor': 'white', 'alpha': 0.5, 'pad': 10})

# plt.legend()
plt.title(f'\n{runtype} run {run}, {rt_min:.2f} mins', fontsize=12)
plt.tight_layout()

In [ ]:
sub_aoe_hist = aoe_hist_norm - cut_aoe_hist_norm

In [ ]:
pcm_sub = plt.pcolormesh(X, Y, sub_aoe_hist, norm=LogNorm(0.001, 0.1))